[(토론-2) OD을 위한 Stratified Group KFold (feat. sklearn)](https://stages.ai/competitions/218/discussion/talk/post/1791)

미리 알면 글을 읽는데 도움이 되는 개념

- Cross-validation : https://velog.io/@jee-9/Scikit-learn-교차-검증Cross-validation-KFold-Stratified-KFold-crossvalscore

Object Detection 은 classification과 다르게 하나의 이미지에 여러개의 class가 존재하기 때문에 Cross-validation(CV)를 나누는 방법이 다릅니다. 따라서 어떻게 CV를 나누면 좋을지 알아보도록 하겠습니다.

![](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000651/files/fe304c35-ba41-4595-b234-f90c0c8c1b62..png)

Object Detection에서는 stratifiedGroupKfold를 주로 활용합니다.

stratifiedGroupKfold는 class를 group 별로 묶어서 class별 분포를 유사하게 나누는 방법입니다. 여기서 group은 Object Detection에서 하나의 image안에 여러개의 class가 존재할 수 있으므로 image별로 group을 나눠주면 됩니다. 그렇게 되면 이미지별로 group을 묶었기 때문에 stratifiedKFold를 사용할 수 있게 됩니다.

출처 : https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py

필요한 파라미터

![](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000651/files/f6231a02-ffbe-44de-8cd3-5d31d0a7f877..png)

In [ ]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

# load json annotation = {dataset file 경로}

with open(annotation) as f:
    data = json.load(f)

var = [(ann["image_id"], ann["category_id"]) for ann in data["annotations"]]
X = np.ones((len(data["annotations"]), 1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])

그리고 실제로 데이터가 잘 나누어졌는지는 아래의 코드를 통해 확인 했습니다! (아래 코드는 캐글 노트북 - https://www.kaggle.com/code/jakubwasikowski/stratified-group-k-fold-cross-validation/notebook 을 참고하였습니다~)

In [ ]:
# check distribution

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    # train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0 distrs.append(get_distribution(train_y))

    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

categories = [d['name'] for d in data['categories']]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

데이터셋을 나눈 뒤, 데이터셋이 잘 나누어졌는지 확인하는 과정도 필요합니다. 위의 코드를 사용해서 데이터셋이 잘 나누어졌는지 확인해보세요.

위의 방식 이외에도 어떤 식으로 데이터셋을 Split하면 좋을지, 우리 팀은 어떤 전략을 사용했는지 공유해주세요. : )